In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

from sklearn.utils import shuffle
from sklearn.cross_validation import train_test_split

In [ ]:
from sklearn.metrics import f1_score
from sklearn.datasets import fetch_mldata
mnist = fetch_mldata('MNIST original')

In [ ]:
mnist = fetch_mldata('MNIST original')
mnist_x, mnist_y = mnist.data.astype("float32")/255.0, mnist.target.astype("int32")

train_X, test_X, train_y, test_y = train_test_split(mnist_x, mnist_y, test_size=0.2)

Denoising auto-encode

In [ ]:
import numpy as np
import theano
import theano.tensor as T
from theano.tensor.shared_randomstreams import RandomStreams
from collections import OrderedDict

In [ ]:
rng = np.random.RandomState(1234)
theano_rng = RandomStreams(rng.randint(1234))

In [ ]:
def sgd(params,gparams,lr=0.1):
    updates = OrderedDict()
    for param, gparam in zip(params, gparams):
        updates[param] = param - lr * gparam
    return updates

Autoencoderクラス

In [ ]:
class Autoencoder:
    def __init__(self,visible_dim,hidden_dim,function):
        ## WRITE ME
        self.a = 
        self.b = 
        self.params = []
        
    #encoder
    def encode(self,x):
        ## WRITE ME
        return y
    
    #decoder
    def decode(self,x):
        ## WRITE ME
        return y
    
    #forward propagation
    def prop(self,x):
        ## WRITE ME
        return reconst_x
    
    #reconstruction error
    def reconst_error(self,x,noise):
        ## WRITE ME
        return error, reconst_x

In [ ]:
model = Autoencoder(train_X.shape[1],500,T.nnet.sigmoid)

x = T.matrix('x')
noise = T.matrix('noise')

cost,reconst_x = model.reconst_error(x,noise)
params  = model.params
gparams = T.grad(cost, params)
updates = sgd(params,gparams) 

train = theano.function([x,noise], [cost,reconst_x], updates=updates, allow_input_downcast=True)

In [ ]:
corruption_level = 0.3
batch_size = 100
nbatches = train_X.shape[0] // batch_size

for epoch in range(10):
    train_X = shuffle(train_X)
    err_all=[]
    for i in range(0,nbatches):
        start = i * batch_size
        end   = start + batch_size
        
        noise = rng.binomial(size=train_X[start:end].shape, n=1, p=1-corruption_level)
        err,reconst_x = train(train_X[start:end],noise)
        err_all.append(err)
    print "Epoch:%d, Error:%lf" %(epoch, np.mean(err))

In [ ]:
weight = model.W.get_value(borrow=True).T

fig = plt.figure(figsize=(10, 10))
for i in range(100):
    ax = fig.add_subplot(10, 10, i + 1, xticks=[], yticks=[])
    ax.imshow(weight[i].reshape((28,28)), cmap='gray')

RBM

In [ ]:
rng = np.random.RandomState(1234)
theano_rng = RandomStreams(rng.randint(1234))

RBMクラスの実装

In [ ]:
class RBM:
    def __init__(self,visible_dim,hidden_dim,function,k):
        ## WRITE ME
        self.a = 
        self.b = 
        
        self.params = []
        
    def propup(self,input):
        ## WRITE ME
        return output
    
    def propdown(self,input):
        ## WRITE ME
        return output

    #p(h|v)
    def ph_v(self,v):
        ## WRITE ME
        return h_sample,h
    
    #p(v|h)
    def pv_h(self,h):
        ## WRITE ME
        return v_sample,v
    
    #gibbs sampling(h→v→h)
    def gibbs_hvh(self,h):
        ## WRITE ME
        return h_sample,mean_h
        
    #gibbs sampling(v→h→v)
    def gibbs_vhv(self,v):
        ## WRITE ME
        return v_sample,mean_v
    
    #cost(free energy)
    def free_energy(self,input):
        y = T.dot(input,self.W) + self.b
        return -T.dot(input,self.a) -T.sum(T.log(1 + T.exp(y)),axis=1)
    
    #reconstruction error
    def reconst_error(self,v):
        v_sample = v
        for k in range(self.k):
            v_sample,mean_v = model.gibbs_vhv(v_sample)
            
        cross_entropy = ## WRITE ME
        return cross_entropy

In [ ]:
batch_size = 100
nbatches = train_X.shape[0] // batch_size

persistent = train_X.copy()
for epoch in range(10):
    train_X,persistent = shuffle(train_X,persistent)
    err_all=[]
    for i in range(0,nbatches):
        start = i * batch_size
        end   = start + batch_size
        
        cost,_x = train(train_X[start:end],persistent[start:end])
        persistent[start:end]=_x
        err = reconst(train_X[start:end])
        err_all.append(err)
    print "Epoch:%d, Error:%lf" %(epoch, np.mean(err))

重みの可視化

In [ ]:
weight = model.W.get_value(borrow=True).T

fig = plt.figure(figsize=(10, 10))
for i in range(100):
    ax = fig.add_subplot(10, 10, i + 1, xticks=[], yticks=[])
    ax.imshow(weight[i].reshape((28,28)), cmap='gray')

データの生成

In [ ]:
#gibbs sampling
v = T.matrix("v")
[sample_v,mean_v],updates = theano.scan(fn=model.gibbs_vhv,outputs_info=[v,None],n_steps=1000)
sample = theano.function([v], [sample_v[-1],mean_v[-1]],updates=updates,allow_input_downcast=True)

#seed
test_X = shuffle(test_X)
sample_v,mean_v = sample(test_X[0:100])

fig = plt.figure(figsize=(10, 10))
for i in range(100):
    ax = fig.add_subplot(10, 10, i + 1, xticks=[], yticks=[])
    ax.imshow(mean_v[i].reshape((28,28)), cmap='gray')

## Stacked auto-encoder

In [ ]:
import numpy as np
import theano
import theano.tensor as T
from theano.tensor.shared_randomstreams import RandomStreams
from collections import OrderedDict
rng = np.random.RandomState(1234)

%matplotlib inline
import matplotlib.pyplot as plt

from sklearn.utils import shuffle
from sklearn.cross_validation import train_test_split

from sklearn.metrics import f1_score
from sklearn.datasets import fetch_mldata
mnist = fetch_mldata('MNIST original')
mnist_x, mnist_y = mnist.data.astype("float32")/255.0, mnist.target.astype("int32")

以下のセルを完成させて提出

In [ ]:
#Autoencoder (or RBM)
class Autoencoder:
    def __init__(self,visible_dim,hidden_dim,W,function):
        ## WRITE ME
        
    #encoder
    def encode(self,x):
        ## WRITE ME
    
    #decoder
    def decode(self,x):
        ## WRITE ME
    
    #forward propagation
    def prop(self,x):
        ## WRITE ME
    
    #reconstruction error
    def reconst_error(self,x,noise):
        ## WRITE ME
        
#SGD
def sgd(params,gparams,lr=0.1):
    updates = OrderedDict()
    for param, gparam in zip(params, gparams):
        updates[param] = param - lr * gparam
    return updates
    
#Multi Layer Perceptron
class Layer:
    def __init__(self, in_dim, out_dim, function):
        ## WRITE ME
        self.set_pretraining()

    def fprop(self, x):
        ## WRITE ME
    
    def set_pretraining(self):
        ae = Autoencoder(self.in_dim,self.out_dim,self.W,self.function)

        x = T.matrix('x')
        noise = T.matrix('noise')

        cost,reconst_x = ae.reconst_error(x,noise)
        params  = ae.params
        gparams = T.grad(cost, params)
        updates = sgd(params,gparams)

        self.pretraining = theano.function([x,noise], [cost,reconst_x], updates=updates, allow_input_downcast=True)
        
        hidden = ae.encode(x)
        self.encode_function = theano.function([x], hidden, allow_input_downcast=True)

train_X, valid_X, train_y, valid_y = train_test_split(mnist_x, mnist_y, test_size=0.2, random_state=42)

layers = [## WRITE ME
]

#Pre-training
X = train_X
for l, layer in enumerate(layers[:-1]):
    corruption_level = ## WRITE ME
    batch_size = 100
    nbatches = X.shape[0] // batch_size

    for epoch in range(10):
        ## WRITE ME
        print "Pre-training:: layer:%d, Epoch:%d, Error:%lf" %(l,epoch, np.mean(err))
    X = layer.encode_function(X)

#Fine-tuning
x, t = T.fmatrix("x"), T.ivector("t")
params = []
for i, layer in enumerate(layers):
    params += layer.params
    if i == 0:
        layer_out = layer.fprop(x)
    else:
        layer_out = layer.fprop(layer_out)

y = layers[-1].h
cost = - T.mean((T.log(y))[T.arange(x.shape[0]), t])

gparams = T.grad(cost, params)
updates = sgd(params,gparams)

train = theano.function([x,t], cost, updates=updates)
valid  = theano.function([x,t],[cost, T.argmax(y, axis=1)])
test  = theano.function([x],T.argmax(y, axis=1))

batch_size = 100
nbatches = train_X.shape[0]//batch_size
for epoch in range(50):
    ## WRITE ME
    print "EPOCH:: %i, Validation cost: %.3f, Validation F1: %.3f"%(epoch+1, valid_cost, f1_score(valid_y, pred, average="macro"))

#pred_y = test(test_X)